<a href="https://colab.research.google.com/github/dai-zz-flora/something_about_llm/blob/main/llm_reading_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 说明
本脚本展示的是如何使用transformer,faiss,sentence-transformers库来调用大语言模型来分析PDF文档，根据文档内容进行问答

本文的大语言模型使用的是清华的chatglm，版本是THUDM/chatglm-6b-int4



# 准备工作


## 1. 检测GPU

In [ ]:
!nvidia-smi

Thu May 25 14:34:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. 安装必要的python库

本文会用到transformer、faiss等比较有名的python库

In [ ]:
!apt -y update -qq
!apt -y install -qq aria2
!pip install -q transformers datasets  sentencepiece cpm_kernels mdtex2html accelerate sentencepiece install faiss-gpu  sentence-transformers  PyPDF2 pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.0 MB/s eta 0:00:00


## 3. 下载测试pdf文档

In [ ]:
%cd /content
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://ask.qcloudimg.com/draft/1103456/9xw693yh19.pdf" -d /content/ -o "腾讯云区块链TBaaS产品白皮书.pdf"

/content

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
1fe5aa|OK  |   584KiB/s|/content//腾讯云区块链TBaaS产品白皮书.pdf

Status Legend:
(OK):download completed.


## 4. 定义一个pdf解析函数

In [ ]:
import PyPDF2
import pdfplumber
import pandas as pd


def parse_pdf(pdf_path):
    text_pages = []
    images = []
    tables = []

    # parse text from pdf
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        for i in range(num_pages):
            page = reader.pages[i]
            text = page.extract_text()
            text_pages.append(text)

    

    # get images and tables from pdf
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 解析图片
            extracted_images = page.images
            images.extend(extracted_images)

            # 解析表格
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                df = pd.DataFrame(table[1:], columns=table[0])
                tables.append(df)

    return text_pages, images, tables

## 5. 初始化模型

这里我们使用量化后的模型THUDM/chatglm-6b-int4，减少内存占用

In [ ]:
from transformers import AutoTokenizer, AutoModel
mode_name='THUDM/chatglm-6b-int4'
tokenizer = AutoTokenizer.from_pretrained(mode_name, trust_remote_code=True)
model = AutoModel.from_pretrained(mode_name, trust_remote_code=True).half().cuda()

No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


# PDF解析




## 1. 解析PDF

In [ ]:
parsed_text, parsed_images, parsed_tables = parse_pdf("/content/腾讯云区块链TBaaS产品白皮书.pdf")

## 2. 生成文本向量

使用 [sentence transformers库](https://www.sbert.net/) 来生成基于词嵌入的文本向量，选用的模型是 [GanymedeNil/text2vec-large-chinese](https://huggingface.co/GanymedeNil/text2vec-large-chinese)

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
embeddings_model = SentenceTransformer("shibing624/text2vec-base-chinese")
# Compute embeddings for all senteces in the corpus
embeddings_corpus = embeddings_model.encode(parsed_text, show_progress_bar=True)

# Change data type of embeddings
embeddings = np.array([embedding for embedding in embeddings_corpus]).astype("float32")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

##3. 利用faiss库建立向量空间索引

In [ ]:

# Instantiate the index with faiss
index = faiss.IndexFlatL2(embeddings.shape[1])

# # Pass the index to IndexIDMap
# index = faiss.IndexIDMap(index
index.add(embeddings)

# 使用ChatGLM-6B来构建对话模型


## 1. 初始化问题，并在向量空间索引中搜寻相关章节

2.1. 生成问题向量

2.2 搜寻前5个相似的章节

2.3 将章节拼接为上下文

In [ ]:
question = '白皮书都讲了什么'
question_vector = embeddings_model.encode([question])

# 查询最相似的文本
k = 5  # 返回最近邻的数量
_, indices = index.search(np.array(question_vector).astype("float32"), k)

# 获取最相关的文本
related_docs = [parsed_text[idx] for idx in indices[0]]
# 拼接上下文
context = "\n".join([content for content in related_docs])
# for text in related_docs:
#   print(text)

## 2. 根据问题模板生成问题

In [ ]:
PROMPT_TEMPLATE = """已知信息：
{context} 

根据上述已知信息，简洁和专业的来回答用户的问题。如果无法从中得到答案，请说 “根据已知信息无法回答该问题” 或 “没有提供足够的相关信息”，不允许在答案中添加编造成分，答案请使用中文。 问题是：{question}"""

prompt = PROMPT_TEMPLATE.replace("{question}", question).replace("{context}", context)

## 3. 交给模型解答问题

In [ ]:
import torch
max_token = 10000
temperature = 0.8
top_p = 0.9
history = []

response, _ = model.chat(tokenizer,prompt,history=[],max_length=max_token,temperature=temperature,top_p=top_p,)


print(response)
print("\n相关文档如下\n")
for text in related_docs:
  print(text)
  print("\n")

本白皮书主要介绍了区块链技术和应用，包括其起源、发展、应用领域和未来发展。

1. 区块链的起源和发展

区块链是一种基于去中心化技术的新型互联网协议，可以让数据在网络中分布式存储和传输，具有去中介化、不可篡改、安全性高等特点。区块链技术最早由中本聪在2009年提出，目前已经成为互联网的重要组成部分。

2. 区块链的应用

区块链可以用于多种应用，包括数字货币、智能合约、供应链管理、物联网等。其中，数字货币是区块链技术最为广泛的应用之一。

3. 区块链的应用领域

区块链在数字货币中的应用非常广泛，如比特币、以太坊等。同时，区块链还可以用于智能合约、供应链管理、物联网等领域。

4. 区块链的未来发展

随着区块链技术的不断发展和成熟，未来区块链将广泛应用于金融、医疗、物流等领域。同时，区块链技术也将成为未来互联网的重要基础设施。

相关文档如下

อ࿟ᄉ౵ॶ৽5#BB4Ӂ௖Ϣ௃඀
三、对腾讯云区块链（TBaaS）开发平台的介绍。首先介绍了腾讯云区块链诞生的原
因，并对其设计原则进行了详细讲解， 然后说明了其能够为行业带来的不同服务能力， 最后对
其后台技术架构进行了相关介绍。
四、 基于腾讯云区块链 （TBaaS） 的应用场景及解决方案。 提出了包括在共享账本、 公证与
记录、 互助保障、 数字资产 、 防伪溯源、 物联网、 共享经济等应用场景能够解决的业务痛点， 并分别枚举了相关案例进行阐述说明。
五、 对区块链未来应用的展望。 基于现有的区块链应用场景， 对区块链的多平台互通、 链外
信任机制， 以及其在未来能够带来的更大价值进行了开放式的展望和创想。
                                                                                          腾讯云区块链团队                                                                                             2018年3月30日前言
每一次产业变革的浪潮都带给世界全新的机遇与想象。
区块链技术的诞生，真正意义上为数字经济提供了全新的价值转移通道，其应用场景几
乎不可限量。工信部于2016年正式发布的《中国区块链技术和应用发展白皮书》，充